In [ ]:
# Linux Package Installation
!apt-get -qq -y install fonts-nanum > /dev/null

# Delete Cache
!rm -rf ~/.cache/matplotlib -rf

In [ ]:
import os

os.kill(os.getpid(), 9)

In [1]:
from collections import Counter
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import numpy as np
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score, f1_score
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

mpl.rcParams['axes.unicode_minus'] = False
plt.rcParams['font.family'] = 'NanumGothic'

Mounted at /content/drive


In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

# 데이터 불러오기

In [6]:
import joblib
tokenizer_Credit_Reporting = joblib.load('/content/drive/MyDrive/Credit_tokenizer.joblib')
model_Credit_Reporting = load_model('/content/drive/MyDrive/best_model_Credit_reporting')

In [7]:
model_Credit_Reporting.summary()

Model: "Conv1D"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 100, 250)          2500000   
                                                                 
 conv1d_3 (Conv1D)           (None, 96, 64)            80064     
                                                                 
 max_pooling1d_3 (MaxPoolin  (None, 48, 64)            0         
 g1D)                                                            
                                                                 
 conv1d_4 (Conv1D)           (None, 44, 128)           41088     
                                                                 
 max_pooling1d_4 (MaxPoolin  (None, 22, 128)           0         
 g1D)                                                            
                                                                 
 conv1d_5 (Conv1D)           (None, 18, 256)           16409

In [3]:
DF = pd.read_csv('/content/drive/MyDrive/Final.csv')
DF['product'] = DF['product'].replace('Money transfer, virtual currency, or money service', 'Money transfer')
DF= DF[['consumer complaint narrative','product','issue']]
DF

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

# 데이터 제품별 분류

In [ ]:
DF['product'].value_counts()

product
Credit reporting               849651
Debt collection                167471
Credit card                     95113
Checking or savings account     77697
Mortgage                        66689
Student loan                    22902
Money transfer                  21188
Vehicle loan or lease           17487
Payday loan                      8343
Name: count, dtype: int64

In [ ]:
DF_Credit = DF[DF['product'] == 'Credit reporting']
DF_Credit

,consumer complaint narrative,product,issue
143845,I sent a letter regarding inaccurate and unkno...,Credit reporting,Problem with a company's investigation into an...
143846,Equifax has refused to delete the negative inq...,Credit reporting,Problem with a company's investigation into an...
143847,Every time I try to delete hard inquiries on m...,Credit reporting,Incorrect information on your report
143848,Every time I try to delete hard inquiries on m...,Credit reporting,Incorrect information on your report
143849,After sending a lot of Dispute Letters and FTC...,Credit reporting,Incorrect information on your report
...,...,...,...
993491,XXXX refuses to take off XXXX accounts that is...,Credit reporting,Incorrect information on your report
993492,I XXXX : XXXX am submitting this complaint by ...,Credit reporting,Improper use of your report
993493,TRANSUNION is in violation of my consumer righ...,Credit reporting,Improper use of your report
993494,Despite my repeated efforts to rectify this si...,Credit reporting,Incorrect information on your report


In [ ]:
DF_Debt = DF[DF['product'] == 'Debt collection']
DF_Debt['issue'].unique()

array(['Written notification about debt',
       'Attempts to collect debt not owed',
       'Took or threatened to take negative or legal action',
       'False statements or representation', 'Communication tactics'],
      dtype=object)

In [ ]:
DF_Card = DF[DF['product'] == 'Credit card']
DF_Card['issue'].unique()

array(['Problem with a purchase shown on your statement',
       'Getting a credit card', 'Closing your account',
       'Other features, terms, or problems', 'Trouble using your card',
       'Fees or interest',
       "Problem with a company's investigation into an existing problem",
       'Problem when making payments',
       'Advertising and marketing, including promotional offers'],
      dtype=object)

In [ ]:
DF_Account = DF[DF['product'] == 'Checking or savings account']
DF_Account['issue'].unique()

array(['Closing an account', 'Problem caused by your funds being low',
       'Managing an account',
       'Problem with a lender or other company charging your account',
       'Opening an account'], dtype=object)

In [ ]:
DF_Mortgage = DF[DF['product'] == 'Mortgage']
DF_Mortgage['issue'].unique()

array(['Struggling to pay mortgage', 'Trouble during payment process',
       'Applying for a mortgage or refinancing an existing mortgage',
       'Closing on a mortgage'], dtype=object)

In [ ]:
DF_Student = DF[DF['product'] == 'Student loan']
DF_Student['issue'].unique()

array(['Dealing with your lender or servicer',
       'Struggling to repay your loan',
       'Incorrect information on your report'], dtype=object)

In [ ]:
DF_Transfer = DF[DF['product'] == 'Money transfer']
DF_Transfer['issue'].unique()

array(['Fraud or scam', 'Other transaction problem',
       'Managing, opening, or closing your mobile wallet account'],
      dtype=object)

In [ ]:
DF_Vehicle = DF[DF['product'] == 'Vehicle loan or lease']
DF_Vehicle['issue'].unique()

array(['Managing the loan or lease', 'Struggling to pay your loan',
       'Problems at the end of the loan or lease'], dtype=object)

In [ ]:
DF_Payday = DF[DF['product'] == 'Payday loan']
DF_Payday['issue'].unique()

array(["Charged fees or interest you didn't expect",
       'Struggling to pay your loan', 'Problem when making payments'],
      dtype=object)

# 자연어처리

In [ ]:
import re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
data = list(DF_Credit['consumer complaint narrative'])
clean = []
for narrative in (data):
 clean_narrative = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…\"\“》XXXX{}]', '', narrative)
 clean.append(clean_narrative)
DF_Credit['narrative'] = clean

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
lower = [text.lower() for text in DF_Credit['narrative'].values]
DF_Credit['narrative'] = lower

In [ ]:
stop_words = set(stopwords.words('english'))
def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    return " ".join(filtered_words)

DF_Credit["narrative"]=DF_Credit["narrative"].apply(lambda x: remove_stopwords(x))

DF_Credit.head()

,consumer complaint narrative,product,issue,narrative
143845,I sent a letter regarding inaccurate and unkno...,Credit reporting,Problem with a company's investigation into an...,sent letter regarding inaccurate unknown thing...
143846,Equifax has refused to delete the negative inq...,Credit reporting,Problem with a company's investigation into an...,equifax refused delete negative inquiries cred...
143847,Every time I try to delete hard inquiries on m...,Credit reporting,Incorrect information on your report,every time try delete hard inquiries report co...
143848,Every time I try to delete hard inquiries on m...,Credit reporting,Incorrect information on your report,every time try delete hard inquiries report co...
143849,After sending a lot of Dispute Letters and FTC...,Credit reporting,Incorrect information on your report,sending lot dispute letters ftc reports negati...


# 토큰화

In [ ]:
def tokenize_nar(nar):
    tokens = word_tokenize(nar)
    #tokens = [token.lower() for token in tokens]
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    return tokens

DF_old['tokenized_nar'] = DF_old['narrative'].apply(tokenize_nar)
print(DF_old['tokenized_nar'])

0        [dear, amex, subject, urgent, followup, disput...
1        [22, amazon, prime, member, opted, take, amazo...
2        [started, 2022, making, monthly, payment, auto...
3        [note, synchrony, return, call, way, get, comm...
4        [attempted, pay, chase, credit, card, person, ...
                               ...                        
53829    [hi, end, bank, america, sent, letter, stated,...
53830    [banked, bank, received, letter, bank, said, f...
53831    [list, charge, withdraw, checking, account, wi...
53832    [victim, fraud, account, husband, son, 1300000...
53833    [past, summer, 2022, credit, karma, advertised...
Name: tokenized_nar, Length: 53834, dtype: object


In [ ]:
from collections import Counter

# 모든 단어를 하나의 리스트로 펼침
words = [word for sentence in DF_old['tokenized_nar'] for word in sentence]

# 각 단어의 빈도를 계산
word_counts = Counter(words)

print(word_counts)

Counter({'account': 100157, 'credit': 89702, 'payment': 65570, 'bank': 56249, 'card': 47919, 'would': 42930, 'loan': 41832, 'time': 36797, 'received': 34774, 'told': 33947, 'report': 32264, 'mortgage': 29596, 'company': 28778, 'information': 28610, 'called': 28508, 'letter': 28087, 'call': 27280, 'one': 26212, 'never': 26029, 'day': 25801, 'sent': 25465, 'year': 25102, 'charge': 25053, 'month': 24805, 'said': 24614, 'number': 24463, 'paid': 23574, 'amount': 23339, 'get': 23319, 'money': 22502, 'check': 22213, 'pay': 22212, 'made': 21578, 'back': 21424, 'phone': 20322, 'also': 20030, 'could': 19295, 'balance': 19036, 'service': 18929, 'due': 18781, 'well': 18426, 'debt': 17753, 'email': 17669, 'complaint': 17070, 'since': 16967, 'u': 16733, 'statement': 16677, 'fee': 16116, 'interest': 15769, 'fraud': 15523, 'asked': 15368, 'new': 15201, 'home': 14922, 'late': 14895, 'customer': 14341, 'date': 14104, 'still': 13667, 'help': 13428, 'make': 13426, 'reporting': 13175, 'dispute': 13151, 'na

In [ ]:
sentences = []
for review in list(DF_old['narrative']):
    sentences.append(review.split())
sentences

In [ ]:
DF_old

,consumer complaint narrative,product,narrative,product_L,tokenized_nar
0,Dear Amex Subject : Urgent Follow-Up - Dispute...,Credit card,dear amex subject urgent followup dispute rega...,1,"[dear, amex, subject, urgent, followup, disput..."
1,Being a 22+ Amazon Prime member I opted to tak...,Credit card,22 amazon prime member opted take amazon store...,1,"[22, amazon, prime, member, opted, take, amazo..."
2,this all started in XX/XX/2022 I had been maki...,Credit card,started 2022 making monthly payments auto pay ...,1,"[started, 2022, making, monthly, payment, auto..."
3,XX/XX/XXXX Note : Synchrony will not return ou...,Credit card,note synchrony return call way get communicate...,1,"[note, synchrony, return, call, way, get, comm..."
4,On XX/XX/XXXX I attempted to pay my Chase Cred...,Credit card,attempted pay chase credit card person locatio...,1,"[attempted, pay, chase, credit, card, person, ..."
...,...,...,...,...,...
53829,Hi.\n\nAt end of XX/XX/XXXX Bank of America s...,Checking or savings account,hi end bank america sent letter stated account...,0,"[hi, end, bank, america, sent, letter, stated,..."
53830,"In XXXX, the XXXX XXXX XXXX banked with XXXX B...",Checking or savings account,banked bank received letter bank said future p...,0,"[banked, bank, received, letter, bank, said, f..."
53831,Below is a list of charges that was withdraw f...,Checking or savings account,list charges withdraw checking account without...,0,"[list, charge, withdraw, checking, account, wi..."
53832,"I was a Victim of Fraud my Account, Husband an...",Checking or savings account,victim fraud account husband son 13000000 wipe...,0,"[victim, fraud, account, husband, son, 1300000..."


In [ ]:
DF_old['tokenized_nar']

0        [dear, amex, subject, urgent, followup, disput...
1        [22, amazon, prime, member, opted, take, amazo...
2        [started, 2022, making, monthly, payment, auto...
3        [note, synchrony, return, call, way, get, comm...
4        [attempted, pay, chase, credit, card, person, ...
                               ...                        
53829    [hi, end, bank, america, sent, letter, stated,...
53830    [banked, bank, received, letter, bank, said, f...
53831    [list, charge, withdraw, checking, account, wi...
53832    [victim, fraud, account, husband, son, 1300000...
53833    [past, summer, 2022, credit, karma, advertised...
Name: tokenized_nar, Length: 53834, dtype: object

# 인코딩

## 1.레이블인코딩

In [ ]:
from sklearn.preprocessing import LabelEncoder
DF['issue_L'] = LabelEncoder().fit_transform(DF['issue'])

In [ ]:
DF['issue_L'].value_counts()

## 2.원-핫 인코딩

In [ ]:
A = pd.get_dummies(DF_Credit['issue']).astype('int')
DF_Credit = pd.concat([DF_Credit, A], axis = 1)
DF_Credit.head()

,consumer complaint narrative,product,issue,narrative,Credit monitoring or identity theft protection services,Improper use of your report,Incorrect information on your report,Problem with a company's investigation into an existing problem,Problem with fraud alerts or security freezes,Unable to get your credit report or credit score
143845,I sent a letter regarding inaccurate and unkno...,Credit reporting,Problem with a company's investigation into an...,sent letter regarding inaccurate unknown thing...,0,0,0,1,0,0
143846,Equifax has refused to delete the negative inq...,Credit reporting,Problem with a company's investigation into an...,equifax refused delete negative inquiries cred...,0,0,0,1,0,0
143847,Every time I try to delete hard inquiries on m...,Credit reporting,Incorrect information on your report,every time try delete hard inquiries report co...,0,0,1,0,0,0
143848,Every time I try to delete hard inquiries on m...,Credit reporting,Incorrect information on your report,every time try delete hard inquiries report co...,0,0,1,0,0,0
143849,After sending a lot of Dispute Letters and FTC...,Credit reporting,Incorrect information on your report,sending lot dispute letters ftc reports negati...,0,0,1,0,0,0


# 패딩

pad_sequences( )

padding = 'pre': 문장 길이 2500으로 변경 후 앞쪽에 '0' 채우기

padding = 'post': 문장 길이 2500으로 변경 후 뒤쪽에 '0' 채우기

maxlen: 최대 길이 지정 (미지정 시 가장 긴 문장 길이)

truncating: maxlen보다 긴 문장 값 제거 ('pre' or 'post')

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words = 10000)
tokenizer.fit_on_texts(DF_Credit['narrative'])

In [ ]:
import joblib
joblib.dump(tokenizer, 'tokenizer.joblib')

['tokenizer.joblib']

In [ ]:
A = tokenizer.word_index
len(A)

240745

In [ ]:
sequences = tokenizer.texts_to_sequences(DF_Credit['narrative'])
len(sequences)

849651

In [ ]:
print('리뷰 최대 길이 :', max(len(L) for L in sequences))
print('리뷰 평균 길이 :', sum(map(len, sequences))/len(sequences))

리뷰 최대 길이 : 2922
리뷰 평균 길이 : 72.02427938059273


In [ ]:
print(sequences[0])
print
print(DF['narrative'][0])

[1059, 1688, 874, 2998, 2111, 73, 217, 59, 187, 2228, 22, 216, 745, 1059, 503, 594, 19, 433, 674, 73, 217, 59, 187, 80, 1665, 22, 216, 373, 2080, 99, 14, 1027, 2615, 11, 294, 8, 8, 61, 4096, 257, 92, 62, 501, 28, 660, 1110, 73, 950, 144, 86, 30, 8, 69, 22, 165, 102, 64, 34, 122, 274, 41, 2624, 285, 22, 506, 2200, 572, 19, 1066, 6192, 357, 1481, 22, 772, 301, 380, 368, 4303, 587, 290, 457, 15, 445, 357, 690, 818, 2700, 237, 1343, 1263, 217, 59, 1317, 12, 72, 52, 208, 142, 783, 303, 190, 578, 60, 1017, 15, 1109, 4134, 170, 1360, 111, 14, 32, 279, 553, 325, 45, 170, 2081, 9869, 2945, 507, 180, 79, 287, 465, 491, 2657, 11, 8, 122, 457, 1762, 73, 5043, 1003, 340, 92, 1360, 111, 2442, 122, 5044, 2701, 2120, 507, 622, 32, 27, 302, 446, 1438, 912, 235, 719, 794, 273, 572, 275, 132, 976, 6985, 229, 14, 413, 1718, 119]
dear amex subject urgent followup dispute regarding charges unperformed services ref letter dated fax dear american express company writing follow dispute regarding charges unperf

In [ ]:
max_len = max(len(s) for s in sequences)

max_len

2922

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

sequences = pad_sequences(sequences,
                        maxlen = 100,
                        padding = 'post',
                        truncating = 'post')

# 임베딩

In [ ]:
#먼저 패딩을 하고 실행시킬것

In [ ]:
import tensorflow_hub as hub

embeddings_index = hub.load('https://tfhub.dev/google/Wiki-words-250/2')

In [ ]:
embeddings_index(['apple'])

In [ ]:
%%time
max_words = 10000
embedding_dim = 250
word_index = tokenizer.word_index

embedding_matrix = np.zeros((max_words, embedding_dim))

for word, i in word_index.items():
    embedding_vector = embeddings_index([word])
    if i < max_words:
        if embedding_vector is not None:
            # 임베딩 인덱스에 없는 단어는 모두 0이 됨
            embedding_matrix[i] = embedding_vector

CPU times: user 3min 6s, sys: 13.9 s, total: 3min 20s
Wall time: 2min 34s


In [ ]:
embedding_matrix.shape

(10000, 250)

# 데이터 Split

In [5]:
from tensorflow.keras.datasets import reuters
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

In [ ]:
from tensorflow.keras.models import load_model
model = load_model('/content/drive/MyDrive/best_model')

In [ ]:
#DF_Credit
y =DF_Credit[['Credit monitoring or identity theft protection services',
       'Improper use of your report', 'Incorrect information on your report',
       "Problem with a company's investigation into an existing problem",
       'Problem with fraud alerts or security freezes',
       'Unable to get your credit report or credit score']].values

In [ ]:
y = y.astype(np.float32)
y

array([[0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       ...,
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.]], dtype=float32)

In [ ]:
X = sequences
Y = y
print("\n 테스트 정확도: %.4f" % (model.evaluate(X, Y)[1]))

41455/41455 [==============================] - 182s 4ms/step - loss: 0.2192 - accuracy: 0.9302

 테스트 정확도: 0.9302


In [ ]:
#x = X_tfidf
#x = X_Count
#x = test_data_vecs
x = sequences
#Y = DF_old['product_L']
Y = y
x_train, x_test, y_train, y_test = train_test_split(x, Y,test_size = 0.1, stratify = Y)
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(764685, 100) (764685, 6) (84966, 100) (84966, 6)


In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(x_train, y_train,
                                                      test_size = 0.2,
                                                      stratify = y_train,
                                                      random_state = 2045)
print(X_train.shape, X_valid.shape, y_train.shape, y_valid.shape)

(611748, 100) (152937, 100) (611748, 6) (152937, 6)


# 모델링

# 임베딩 적용

In [ ]:
model.layers[0].set_weights([embedding_matrix])

In [ ]:
model.layers[0].trainable = False

# 모델 CheckPoint

In [ ]:
## early stopping하는법
#es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
# 검증 데이터 손실(val_loss)이 증가하면, 과적합 징후므로 검증 데이터 손실이 4회 증가하면 학습을 조기 종료(Early Stopping)
mc = ModelCheckpoint('best_model', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

# 모델선택

## 1

In [ ]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import regularizers
model = models.Sequential(name = 'Conv1D')
model.add(layers.Embedding(input_dim = 10000,
                          output_dim = 128,
                          input_length = 100))
model.add(layers.Conv1D(32, 5, activation = 'relu'))
model.add(layers.MaxPooling1D(2))
model.add(layers.Conv1D(64, 5, activation = 'relu'))
model.add(layers.MaxPooling1D(2))
model.add(layers.Conv1D(128, 5, activation = 'relu'))
model.add(layers.MaxPooling1D(2))
model.add(layers.LSTM(150))
model.add(layers.Dense(6, activation = 'softmax'))

In [ ]:
model.compile(loss = 'categorical_crossentropy',
             optimizer = 'adam',
             metrics = ['accuracy'])

In [ ]:
%%time

Hist_KKK = model.fit(X_train, y_train,
                       callbacks=[mc],
                       epochs = 20,
                       batch_size = 512,
                       validation_data = (X_valid,y_valid))

In [ ]:
model = load_model('best_model')

In [ ]:
print("\n 테스트 정확도: %.4f" % (model.evaluate(x_test, y_test)[1]))
# model = models.Sequential(name = 'Conv1D')
# model.add(layers.Embedding(input_dim = 10000,
#                           output_dim = 128,
#                           input_length = 100))
# model.add(layers.Conv1D(32, 5, activation = 'relu'))
# model.add(layers.MaxPooling1D(2))
# model.add(layers.Conv1D(64, 5, activation = 'relu'))
# model.add(layers.MaxPooling1D(2))
# model.add(layers.Conv1D(128, 5, activation = 'relu'))
# model.add(layers.MaxPooling1D(2))
# model.add(layers.LSTM(120))
# model.add(layers.Dense(6, activation = 'softmax'))

2656/2656 [==============================] - 19s 6ms/step - loss: 0.6509 - accuracy: 0.8457

 테스트 정확도: 0.8457


In [ ]:
print("\n 테스트 정확도: %.4f" % (model.evaluate(x_test, y_test)[1]))
# model = models.Sequential(name = 'Conv1D')
# model.add(layers.Embedding(input_dim = 10000,
#                           output_dim = 128,
#                           input_length = 100))
# model.add(layers.Conv1D(32, 5, activation = 'relu'))
# model.add(layers.MaxPooling1D(2))
# model.add(layers.Conv1D(64, 5, activation = 'relu'))
# model.add(layers.MaxPooling1D(2))
# model.add(layers.LSTM(120))
# model.add(layers.Dense(6, activation = 'softmax'))

5311/5311 [==============================] - 22s 4ms/step - loss: 0.6868 - accuracy: 0.8394

 테스트 정확도: 0.8394


## 2

In [ ]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import regularizers
model = models.Sequential(name = 'Conv1D')
model.add(layers.Embedding(input_dim = 10000,
                          output_dim = 128,
                          input_length = 150))
model.add(layers.Conv1D(32, 5, activation = 'relu'))
model.add(layers.MaxPooling1D(2))
model.add(layers.Conv1D(64, 5, activation = 'relu'))
model.add(layers.MaxPooling1D(2))
model.add(layers.LSTM(64))
model.add(layers.Dense(512, activation = 'relu', kernel_regularizer = regularizers.l2(0.0005)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(256, activation = 'relu', kernel_regularizer = regularizers.l2(0.0005)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(128, activation = 'relu', kernel_regularizer = regularizers.l2(0.0005)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(6, activation = 'softmax'))

In [ ]:
model = models.Sequential(name = 'Conv1D')
model.add(layers.Embedding(input_dim = 10000,
                          output_dim = 128,
                          input_length = 100))
model.add(layers.Conv1D(32, 5, activation = 'relu'))
model.add(layers.MaxPooling1D(2))
model.add(layers.Conv1D(64, 5, activation = 'relu'))
model.add(layers.MaxPooling1D(2))
model.add(layers.LSTM(64))
model.add(layers.Dense(512, activation = 'relu'))
model.add(layers.Dense(256, activation = 'relu'))
model.add(layers.Dense(128, activation = 'relu'))
model.add(layers.Dense(6, activation = 'softmax'))

In [ ]:
model.compile(loss = 'categorical_crossentropy',
             optimizer = 'adam',
             metrics = ['accuracy'])

In [ ]:
%%time

Hist_KKK = model.fit(X_train, y_train,
                       epochs = 20,
                       callbacks=[mc],
                       batch_size = 512,
                       validation_data = (X_valid,y_valid))

## 3

In [ ]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import regularizers
model = models.Sequential(name = 'Conv1D')
model.add(layers.Embedding(input_dim = 10000,
                         output_dim = 250,
                         input_length = 100))
model.add(layers.Conv1D(64, 5, activation = 'relu'))
model.add(layers.MaxPooling1D(2))
model.add(layers.Conv1D(128, 5, activation = 'relu'))
model.add(layers.MaxPooling1D(2))
model.add(layers.Conv1D(256, 5, activation = 'relu'))
model.add(layers.MaxPooling1D(2))
model.add(layers.Bidirectional(layers.LSTM(120,return_sequences = True)))
model.add(layers.LSTM(120,return_sequences = False))
model.add(layers.Dense(6, activation = 'softmax'))

In [ ]:
model.compile(loss = 'categorical_crossentropy',
             optimizer = 'adam',
             metrics = ['accuracy'])

In [ ]:
Hist_KKK = model.fit(X_train, y_train,
                     epochs = 20,
                     callbacks=[mc],
                     batch_size = 512,
                    validation_data = (X_valid,y_valid))

Epoch 1/20
1195/1195 [==============================] - ETA: 0s - loss: 0.6815 - accuracy: 0.7283
Epoch 1: val_accuracy did not improve from 0.85601
1195/1195 [==============================] - 91s 70ms/step - loss: 0.6815 - accuracy: 0.7283 - val_loss: 0.5546 - val_accuracy: 0.7837
Epoch 2/20
1195/1195 [==============================] - ETA: 0s - loss: 0.5017 - accuracy: 0.8093
Epoch 2: val_accuracy did not improve from 0.85601
1195/1195 [==============================] - 48s 40ms/step - loss: 0.5017 - accuracy: 0.8093 - val_loss: 0.5094 - val_accuracy: 0.8057
Epoch 3/20
1194/1195 [============================>.] - ETA: 0s - loss: 0.4138 - accuracy: 0.8490
Epoch 3: val_accuracy did not improve from 0.85601
1195/1195 [==============================] - 43s 36ms/step - loss: 0.4138 - accuracy: 0.8490 - val_loss: 0.4891 - val_accuracy: 0.8238
Epoch 4/20
1195/1195 [==============================] - ETA: 0s - loss: 0.3420 - accuracy: 0.8787
Epoch 4: val_accuracy did not improve from 0.85601

In [ ]:
model = load_model('best_model')

In [ ]:
print("\n 테스트 정확도: %.4f" % (model.evaluate(x_test, y_test)[1]))
#임베딩적용 o

2656/2656 [==============================] - 15s 5ms/step - loss: 0.6746 - accuracy: 0.8570

 테스트 정확도: 0.8570


In [ ]:
print("\n 테스트 정확도: %.4f" % (model.evaluate(x_test, y_test)[1]))
#임베딩적용 x

2656/2656 [==============================] - 16s 5ms/step - loss: 0.6746 - accuracy: 0.8570

 테스트 정확도: 0.8570


In [ ]:
print("\n 테스트 정확도: %.4f" % (model.evaluate(x_test, y_test)[1]))
model = models.Sequential(name = 'Conv1D')
model.add(layers.Embedding(input_dim = 10000,
                         output_dim = 250,
                         input_length = 100))
model.add(layers.Conv1D(64, 5, activation = 'relu'))
model.add(layers.MaxPooling1D(2))
model.add(layers.Conv1D(128, 5, activation = 'relu'))
model.add(layers.MaxPooling1D(2))
model.add(layers.Conv1D(256, 5, activation = 'relu'))
model.add(layers.MaxPooling1D(2))
model.add(layers.Bidirectional(layers.LSTM(120,return_sequences = True)))
model.add(layers.LSTM(120,return_sequences = False))
model.add(layers.Dense(6, activation = 'softmax'))

2656/2656 [==============================] - 20s 7ms/step - loss: 0.7681 - accuracy: 0.8572

 테스트 정확도: 0.8572


In [ ]:
print("\n 테스트 정확도: %.4f" % (model.evaluate(x_test, y_test)[1]))
# model = models.Sequential(name = 'Conv1D')
# model.add(layers.Embedding(input_dim = 10000,
#                          output_dim = 250,
#                          input_length = 100))
# model.add(layers.Conv1D(64, 5, activation = 'relu'))
# model.add(layers.MaxPooling1D(2))
# model.add(layers.Conv1D(128, 5, activation = 'relu'))
# model.add(layers.MaxPooling1D(2))
# model.add(layers.Conv1D(256, 5, activation = 'relu'))
# model.add(layers.MaxPooling1D(2))
# model.add(layers.LSTM(120,return_sequences = True))
# model.add(layers.LSTM(120,return_sequences = False))
# model.add(layers.Dense(6, activation = 'softmax'))

2656/2656 [==============================] - 13s 5ms/step - loss: 0.5611 - accuracy: 0.7816

 테스트 정확도: 0.7816


In [ ]:
print("\n 테스트 정확도: %.4f" % (model.evaluate(x_test, y_test)[1]))
#best_model_q
# model = models.Sequential(name = 'Conv1D')
# model.add(layers.Embedding(input_dim = 10000,
#                          output_dim = 250,
#                          input_length = 100))
# model.add(layers.Conv1D(64, 5, activation = 'relu'))
# model.add(layers.MaxPooling1D(2))
# model.add(layers.Conv1D(128, 5, activation = 'relu'))
# model.add(layers.MaxPooling1D(2))
# model.add(layers.Conv1D(256, 5, activation = 'relu'))
# model.add(layers.MaxPooling1D(2))
# model.add(layers.Bidirectional(layers.LSTM(120)))
# model.add(layers.Dense(6, activation = 'softmax'))

2656/2656 [==============================] - 15s 5ms/step - loss: 0.8081 - accuracy: 0.8556

 테스트 정확도: 0.8556


In [ ]:
print("\n 테스트 정확도: %.4f" % (model.evaluate(x_test, y_test)[1]))
# model = models.Sequential(name = 'Conv1D')
# model.add(layers.Embedding(input_dim = 10000,
#                          output_dim = 250,
#                          input_length = 100))
# model.add(layers.Conv1D(64, 5, activation = 'relu'))
# model.add(layers.MaxPooling1D(2))
# model.add(layers.Conv1D(128, 5, activation = 'relu'))
# model.add(layers.MaxPooling1D(2))
# model.add(layers.Conv1D(256, 5, activation = 'relu', kernel_regularizer = regularizers.l2(0.0005)))
# model.add(layers.MaxPooling1D(2))
# model.add(layers.Dropout(0.5))
# model.add(layers.LSTM(120))
# model.add(layers.Dense(6, activation = 'softmax'))

2656/2656 [==============================] - 11s 4ms/step - loss: 0.7389 - accuracy: 0.8540

 테스트 정확도: 0.8540


In [ ]:
print("\n 테스트 정확도: %.4f" % (model.evaluate(x_test, y_test)[1]))
#패딩 + 임베딩한 결과
# model = models.Sequential(name = 'Conv1D')
# model.add(layers.Embedding(input_dim = 10000,
#                          output_dim = 250,
#                          input_length = 100))
# model.add(layers.Conv1D(32, 5, activation = 'relu'))
# model.add(layers.MaxPooling1D(2))
# model.add(layers.Conv1D(64, 5, activation = 'relu', kernel_regularizer = regularizers.l2(0.0005)))
# model.add(layers.MaxPooling1D(2))
# model.add(layers.Dropout(0.5))
# model.add(layers.Conv1D(128, 5, activation = 'relu',kernel_regularizer = regularizers.l2(0.0005)))
# model.add(layers.MaxPooling1D(2))
# model.add(layers.Dropout(0.5))
# model.add(layers.LSTM(120))
# model.add(layers.Dense(6, activation = 'softmax'))

2656/2656 [==============================] - 21s 7ms/step - loss: 0.6721 - accuracy: 0.8507

 테스트 정확도: 0.8507


In [ ]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import regularizers
model = models.Sequential(name = 'Conv1D')
model.add(layers.Embedding(input_dim = 10000,
                          output_dim = 250,
                          input_length = 100))
model.add(layers.Conv1D(32, 5, activation = 'relu'))
model.add(layers.MaxPooling1D(2))
model.add(layers.Conv1D(64, 5, activation = 'relu'))
model.add(layers.MaxPooling1D(2))
model.add(layers.Conv1D(128, 5, activation = 'relu'))
model.add(layers.MaxPooling1D(2))
model.add(layers.LSTM(120))
model.add(layers.Dense(6, activation = 'softmax'))

In [ ]:
model.compile(loss = 'categorical_crossentropy',
             optimizer = 'adam',
             metrics = ['accuracy'])

In [ ]:
Hist_KKK = model.fit(X_train, y_train,
                     epochs = 20,
                     callbacks=[mc],
                     batch_size = 512,
                    validation_data = (X_valid,y_valid))

Epoch 1/20
1195/1195 [==============================] - ETA: 0s - loss: 0.6944 - accuracy: 0.7277
Epoch 1: val_accuracy improved from -inf to 0.77826, saving model to best_model
1195/1195 [==============================] - 92s 72ms/step - loss: 0.6944 - accuracy: 0.7277 - val_loss: 0.5717 - val_accuracy: 0.7783
Epoch 2/20
1195/1195 [==============================] - ETA: 0s - loss: 0.5182 - accuracy: 0.8023
Epoch 2: val_accuracy improved from 0.77826 to 0.79874, saving model to best_model
1195/1195 [==============================] - 35s 29ms/step - loss: 0.5182 - accuracy: 0.8023 - val_loss: 0.5261 - val_accuracy: 0.7987
Epoch 3/20
1195/1195 [==============================] - ETA: 0s - loss: 0.4342 - accuracy: 0.8401
Epoch 3: val_accuracy improved from 0.79874 to 0.81834, saving model to best_model
1195/1195 [==============================] - 32s 27ms/step - loss: 0.4342 - accuracy: 0.8401 - val_loss: 0.4966 - val_accuracy: 0.8183
Epoch 4/20
1195/1195 [==============================] -

In [ ]:
model = load_model('best_model')

In [ ]:
print("\n 테스트 정확도: %.4f" % (model.evaluate(x_test, y_test)[1]))
#패딩 + 임베딩한 결과
# model = models.Sequential(name = 'Conv1D')
# model.add(layers.Embedding(input_dim = 10000,
#                           output_dim = 250,
#                           input_length = 100))
# model.add(layers.Conv1D(32, 5, activation = 'relu'))
# model.add(layers.MaxPooling1D(2))
# model.add(layers.Conv1D(64, 5, activation = 'relu'))
# model.add(layers.MaxPooling1D(2))
# model.add(layers.Conv1D(128, 5, activation = 'relu'))
# model.add(layers.MaxPooling1D(2))
# model.add(layers.LSTM(120))
# model.add(layers.Dense(6, activation = 'softmax'))

2656/2656 [==============================] - 12s 4ms/step - loss: 0.6803 - accuracy: 0.8480

 테스트 정확도: 0.8480


# 결과

In [ ]:
y_pred = np.argmax(model.predict(x_test, verbose = 0), axis = 1)

y_pred

array([1, 1, 3, ..., 3, 2, 2])

In [ ]:
y_test = np.argmax(y_test, axis = 1)

In [ ]:
y_pred = model.predict(x_test)

8291/8291 [==============================] - 28s 3ms/step


In [ ]:
print(confusion_matrix(y_test, y_pred))
print('accuracy  : %.4f' % accuracy_score(y_test, y_pred))
print('-' * 20)
print(precision_score(y_test, y_pred, average=None))
print('precision (micro): %.4f' % precision_score(y_test, y_pred, average='micro'))
print('precision (macro): %.4f' % precision_score(y_test, y_pred, average='macro'))
print('precision (weighted): %.4f' % precision_score(y_test, y_pred, average='weighted'))

[[  244    72   184    29    21    27]
 [   15 17432  1854   744    35    29]
 [   39  1326 33442  2665    49    97]
 [   22   798  3474 20617    32    76]
 [    7    36    84    20   455    38]
 [   12    47   179    84    51   630]]
accuracy  : 0.8570
--------------------
[0.71976401 0.88437928 0.85274243 0.85338797 0.70762053 0.70234114]
precision (micro): 0.8570
precision (macro): 0.7867
precision (weighted): 0.8566
